In [1]:
import torch

print("PyTorch Version:", torch.__version__)  
print("CUDA Available:", torch.cuda.is_available())  
print("CUDA Version:", torch.version.cuda)  
print("GPU 개수:", torch.cuda.device_count())  

if torch.cuda.is_available():
    print("GPU 모델명:", torch.cuda.get_device_name(0))  
else:
    print("❌ GPU를 사용할 수 없습니다.")


PyTorch Version: 2.6.0+cu118
CUDA Available: True
CUDA Version: 11.8
GPU 개수: 1
GPU 모델명: NVIDIA GeForce GT 1030


In [ ]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import pandas as pd
import time
from collections import deque

# 1️⃣ CUDA 설정 (GPU가 없으면 CPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2️⃣ 환경 정의 (MarineDebrisEnv)
class MarineDebrisEnv(gym.Env):
    def __init__(self, data):
        super(MarineDebrisEnv, self).__init__()
        
        self.data = data
        self.current_step = 0
        
        # 상태(State) 정의: 선박 밀집도, 해류 속도, 해류 방향
        self.observation_space = gym.spaces.Box(low=0, high=100, shape=(4,), dtype=np.float32)
        
        # 행동(Action) 정의: 상하좌우 4가지 이동
        self.action_space = gym.spaces.Discrete(4)
        
    def reset(self):
        self.current_step = 0
        return self._get_observation()
    
    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        
        # 보상 설계: 쓰레기 밀집도가 높은 곳 방문 시 보상 증가
        reward = self.data.iloc[self.current_step]["Waste_Weight"]
        
        return self._get_observation(), reward, done, {}
    
    def _get_observation(self):
        row = self.data.iloc[self.current_step]
        return np.array([row["avg_ship_density"], row["avg_ship_count"], row["current_speed"], row["current_dir"]], dtype=np.float32)

# 3️⃣ DQN 신경망 모델 (GPU 적용)
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# 4️⃣ DQN 학습 루프 (GPU 최적화 + 진행 상태 모니터링 추가)
def train_dqn(env, episodes=500, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, lr=0.001):
    model = DQN(input_dim=4, output_dim=4).to(device)  # 모델을 GPU로 이동
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    memory = deque(maxlen=10000)
    batch_size = 256  # 배치 크기 증가하여 병렬 학습 최적화
    
    start_time = time.time()  # 전체 학습 시작 시간

    for episode in range(episodes):
        episode_start = time.time()  # 개별 에피소드 시작 시간
        state = env.reset()
        total_reward = 0
        
        for _ in range(len(env.data)):
            # 상태를 텐서로 변환 후 GPU로 이동
            state_tensor = torch.FloatTensor(state).to(device)

            # ε-greedy 정책 적용
            if random.random() < epsilon:
                action = env.action_space.sample()
            else:
                with torch.no_grad():
                    action = torch.argmax(model(state_tensor)).item()
            
            next_state, reward, done, _ = env.step(action)
            memory.append((state, action, reward, next_state, done))
            state = next_state
            total_reward += reward

            if done:
                break
            
            # 경험 재생 (Experience Replay) - GPU 최적화 적용
            if len(memory) >= batch_size:
                batch = random.sample(memory, batch_size)
                states, actions, rewards, next_states, dones = zip(*batch)

                # NumPy 변환 후 GPU로 이동
                states = torch.FloatTensor(np.array(states)).to(device)
                actions = torch.LongTensor(actions).to(device)
                rewards = torch.FloatTensor(rewards).to(device)
                next_states = torch.FloatTensor(np.array(next_states)).to(device)
                dones = torch.FloatTensor(dones).to(device)

                # Q-value 업데이트
                current_q = model(states).gather(1, actions.unsqueeze(1)).squeeze(1)
                next_q = model(next_states).max(1)[0].detach()
                target_q = rewards + gamma * next_q * (1 - dones)

                loss = loss_fn(current_q, target_q)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        
        # ε 값 감소
        epsilon = max(epsilon * epsilon_decay, epsilon_min)
        
        # 🕒 진행 상태 출력 (에피소드별 실행 시간 + 총 경과 시간)
        episode_time = time.time() - episode_start
        total_elapsed = time.time() - start_time
        print(f"Episode {episode}, Total Reward: {total_reward:.2f}, Epsilon: {epsilon:.3f}, Time: {episode_time:.2f}s, Total Elapsed: {total_elapsed:.2f}s")

    print(f"✅ 전체 학습 완료! 총 소요 시간: {time.time() - start_time:.2f}초")
    return model

# 5️⃣ 데이터 로드 및 환경 설정
data = pd.read_csv("marine_data.csv")  # 데이터를 불러와 사용
env = MarineDebrisEnv(data)

# 6️⃣ 모델 훈련 시작
trained_model = train_dqn(env)


Using device: cuda
